In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import json
import bs4
from bs4 import BeautifulSoup
import time

# Race Schedule

In [4]:
schedule = {'season': [],
            'round': [],
            'raceName': [],
            'circuitId': [],
            'circuitName': [],
            'lat': [],
            'long': [],
            'locality': [],
            'country': [],
            'date': [],
            'sprint': [],
            'url': []}

for year in list(range(1950,2023)):
    
    url = 'https://ergast.com/api/f1/{}.json'
    req = requests.get(url.format(year),timeout=None)
    races = req.json()
    
    for item in races['MRData']['RaceTable']['Races']:
        try:
            schedule['season'].append(int(item['season']))
        except:
            schedule['season'].append(None)
            
        try:
            schedule['round'].append(int(item['round']))
        except:
            schedule['round'].append(None)
            
        try:
            schedule['raceName'].append(item['raceName'])
        except:
            schedule['raceName'].append(None)
            
        try:
            schedule['circuitId'].append(item['Circuit']['circuitId'])
        except:
            schedule['circuitId'].append(None)
            
        try:
            schedule['circuitName'].append(item['Circuit']['circuitName'])
        except:
            schedule['circuitName'].append(None)
            
        try:
            schedule['lat'].append(float(item['Circuit']['Location']['lat']))
        except:
            schedule['lat'].append(None)
            
        try:
            schedule['long'].append(float(item['Circuit']['Location']['long']))
        except:
            schedule['long'].append(None)
            
        try:
            schedule['locality'].append(item['Circuit']['Location']['locality'])
        except:
            schedule['locality'].append(None)
            
        try:
            schedule['country'].append(item['Circuit']['Location']['country'])
        except:
            schedule['country'].append(None)
            
        try:
            schedule['date'].append(item['date'])
        except:
            schedule['date'].append(None)
            
        try:
            schedule['sprint'].append(item['Sprint']['date'])
        except:
            schedule['sprint'].append(None)
            
        try:
            schedule['url'].append(item['url'])
        except:
            schedule['url'].append(None)

In [5]:
schedule_df = pd.DataFrame(schedule)
schedule_df.to_csv('raceSchedule.csv')

In [6]:
schedule_df.head()

,season,round,raceName,circuitId,circuitName,lat,long,locality,country,date,sprint,url
0,1950,1,British Grand Prix,silverstone,Silverstone Circuit,52.0786,-1.01694,Silverstone,UK,1950-05-13,None,http://en.wikipedia.org/wiki/1950_British_Gran...
1,1950,2,Monaco Grand Prix,monaco,Circuit de Monaco,43.7347,7.42056,Monte-Carlo,Monaco,1950-05-21,None,http://en.wikipedia.org/wiki/1950_Monaco_Grand...
2,1950,3,Indianapolis 500,indianapolis,Indianapolis Motor Speedway,39.7950,-86.23470,Indianapolis,USA,1950-05-30,None,http://en.wikipedia.org/wiki/1950_Indianapolis...
3,1950,4,Swiss Grand Prix,bremgarten,Circuit Bremgarten,46.9589,7.40194,Bern,Switzerland,1950-06-04,None,http://en.wikipedia.org/wiki/1950_Swiss_Grand_...
4,1950,5,Belgian Grand Prix,spa,Circuit de Spa-Francorchamps,50.4372,5.97139,Spa,Belgium,1950-06-18,None,http://en.wikipedia.org/wiki/1950_Belgian_Gran...


In [7]:
schedule_df.tail()

,season,round,raceName,circuitId,circuitName,lat,long,locality,country,date,sprint,url
1074,2022,18,Japanese Grand Prix,suzuka,Suzuka Circuit,34.8431,136.5410,Suzuka,Japan,2022-10-09,None,http://en.wikipedia.org/wiki/2022_Japanese_Gra...
1075,2022,19,United States Grand Prix,americas,Circuit of the Americas,30.1328,-97.6411,Austin,USA,2022-10-23,None,http://en.wikipedia.org/wiki/2022_United_State...
1076,2022,20,Mexico City Grand Prix,rodriguez,Autódromo Hermanos Rodríguez,19.4042,-99.0907,Mexico City,Mexico,2022-10-30,None,http://en.wikipedia.org/wiki/2022_Mexican_Gran...
1077,2022,21,Brazilian Grand Prix,interlagos,Autódromo José Carlos Pace,-23.7036,-46.6997,São Paulo,Brazil,2022-11-13,2022-11-12,http://en.wikipedia.org/wiki/2022_Brazilian_Gr...
1078,2022,22,Abu Dhabi Grand Prix,yas_marina,Yas Marina Circuit,24.4672,54.6031,Abu Dhabi,UAE,2022-11-20,None,http://en.wikipedia.org/wiki/2022_Abu_Dhabi_Gr...


In [8]:
rounds_df = schedule_df.groupby('season')['round'].apply(list).reset_index(name="rounds")

# Race Results

In [25]:
schedule_df = pd.read_csv('raceSchedule.csv')
rounds_df = schedule_df.groupby('season')['round'].apply(list).reset_index(name="rounds")
results = {'season': [],
          'round': [],
          'circuitId': [],
          'position': [],
          'number': [],
          'driverId': [],
          'constructorId': [],
          'grid': [],
          'time': [],
          'status': [],
          'points': []}
for index in rounds_df.index:
    for rnd_num in rounds_df['rounds'][index]:
        #print(rounds_df['season'][index],rnd_num)
        url = 'http://ergast.com/api/f1/{}/{}/results.json'
        r = requests.get(url.format(rounds_df['season'][index],rnd_num),timeout=None)
        json = r.json()
        try: 
            for item in json['MRData']['RaceTable']['Races'][0]['Results']:
                try:
                    results['season'].append(int(json['MRData']['RaceTable']['Races'][0]['season']))
                except:
                    results['season'].append(None)

                try:
                    results['round'].append(int(json['MRData']['RaceTable']['Races'][0]['round']))
                except:
                    results['round'].append(None)

                try:
                    results['circuitId'].append(json['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId'])
                except:
                    results['circuitId'].append(None)

                try:
                    results['position'].append(int(item['position']))
                except:
                    results['position'].append(None)

                try:
                    results['number'].append(int(item['number']))
                except:
                    results['number'].append(None)

                try:
                    results['driverId'].append(item['Driver']['driverId'])
                except:
                    results['driverId'].append(None)

                try:
                    results['constructorId'].append(item['Constructor']['constructorId'])
                except:
                    results['constructorId'].append(None)

                try:
                    results['grid'].append(int(item['grid']))
                except:
                    results['grid'].append(None)

                try:
                    results['time'].append(item['Time']['time'])
                except:
                    results['time'].append(None)

                try:
                    results['status'].append(item['status'])
                except:
                    results['status'].append(None)

                try:
                    results['points'].append(float(item['points']))
                except:
                    results['points'].append(None)
        except:
            print("Results not available yet")

Results not available yet


In [26]:
results_df = pd.DataFrame(results)
results_df.to_csv('raceResults.csv')

In [27]:
results_df.tail()

,season,round,circuitId,position,number,driverId,constructorId,grid,time,status,points
25362,2022,21,interlagos,16,6.0,latifi,williams,18,+37.038,Finished,0.0
25363,2022,21,interlagos,17,22.0,tsunoda,alphatauri,13,None,+1 Lap,0.0
25364,2022,21,interlagos,18,4.0,norris,mclaren,6,None,Gearbox,0.0
25365,2022,21,interlagos,19,20.0,kevin_magnussen,haas,8,None,Collision,0.0
25366,2022,21,interlagos,20,3.0,ricciardo,mclaren,11,None,Collision,0.0


# Qualifying Results

Part-I: Leveraging F1 official archive to get the qualifying results from the year 1983 to 2002

In [4]:
qualifying_results = pd.DataFrame()
for year in list(range(1983,2003)):
    url = 'https://www.formula1.com/en/results.html/{}/races.html'
    r = requests.get(url.format(year),timeout=None)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    year_links = []
    for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
        link = page.get('href')
        if f'/en/results.html/{year}/races/' in link: 
            year_links.append(link)

    year_df = pd.DataFrame()
    new_url = 'https://www.formula1.com{}'
    for n, link in list(enumerate(year_links)):
        link = link.replace('race-result.html', 'qualifying-0.html')
        df = pd.read_html(new_url.format(link))
        df = df[0]
        df['season'] = year
        df['round'] = n+1
        for col in df:
            if 'Unnamed' in col:
                df.drop(col, axis = 1, inplace = True)

        year_df = pd.concat([year_df, df])

    qualifying_results = pd.concat([qualifying_results, year_df])
    
print(qualifying_results.shape)

(8505, 8)


In [5]:
qualifying_results.tail()

,Pos,No,Driver,Car,Time,season,round,Laps
15,16,12,Olivier Panis PAN,BAR Honda,1:34.192,2002,17,10.0
16,17,17,Pedro de la Rosa DLR,Jaguar Cosworth,1:34.227,2002,17,12.0
17,18,25,Allan McNish MCN,Toyota,1:35.191,2002,17,5.0
18,19,23,Mark Webber WEB,Minardi Asiatech,1:35.958,2002,17,11.0
19,20,22,Alex Yoong YOO,Minardi Asiatech,1:36.267,2002,17,12.0


In [6]:
qualifying_results.to_csv('qualifying_results_I.csv')

Part-II: Utilizing the Ergast Developer API which supports qualifying data from 2003 to current date

In [15]:
schedule_df = pd.read_csv('raceSchedule.csv')
rounds_df = schedule_df.groupby('season')['round'].apply(list).reset_index(name="rounds")
quali_results = {'season': [],
          'round': [],
          'circuitId': [],
          'position': [],
          'number': [],
          'driverId': [],
          'constructorId': [],
          'Q1': [],
          'Q2': [],
          'Q3': []}

for index in rounds_df.loc[rounds_df['season'] >= 2003].index:
    for rnd_num in rounds_df['rounds'][index]:
        #print(rounds_df['season'][index],rnd_num)
        url = 'http://ergast.com/api/f1/{}/{}/qualifying.json'
        r = requests.get(url.format(rounds_df['season'][index],rnd_num),timeout=None)
        json = r.json()
        try: 
            for item in json['MRData']['RaceTable']['Races'][0]['QualifyingResults']:
                try:
                    quali_results['season'].append(int(json['MRData']['RaceTable']['Races'][0]['season']))
                except:
                    quali_results['season'].append(None)

                try:
                    quali_results['round'].append(int(json['MRData']['RaceTable']['Races'][0]['round']))
                except:
                    quali_results['round'].append(None)

                try:
                    quali_results['circuitId'].append(json['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId'])
                except:
                    quali_results['circuitId'].append(None)

                try:
                    quali_results['position'].append(int(item['position']))
                except:
                    quali_results['position'].append(None)

                try:
                    quali_results['number'].append(int(item['number']))
                except:
                    quali_results['number'].append(None)

                try:
                    quali_results['driverId'].append(item['Driver']['driverId'])
                except:
                    quali_results['driverId'].append(None)

                try:
                    quali_results['constructorId'].append(item['Constructor']['constructorId'])
                except:
                    quali_results['constructorId'].append(None)

                try:
                    quali_results['Q1'].append(item['Q1'])
                except:
                    quali_results['Q1'].append(None)

                try:
                    quali_results['Q2'].append(item['Q2'])
                except:
                    quali_results['Q2'].append(None)

                try:
                    quali_results['Q3'].append(item['Q3'])
                except:
                    quali_results['Q3'].append(None)
        except:
            print("Results not available yet")

Results not available yet


In [16]:
quali_results_df = pd.DataFrame(quali_results)
quali_results_df.to_csv('qualifying_results_II.csv')

In [28]:
pd.read_csv('qualifying_results_II.csv').tail()

,Unnamed: 0,season,round,circuitId,position,number,driverId,constructorId,Q1,Q2,Q3
8002,8002,2022,21,interlagos,16,6,latifi,williams,1:15.095,NaN,NaN
8003,8003,2022,21,interlagos,17,24,zhou,alfa,1:15.197,NaN,NaN
8004,8004,2022,21,interlagos,18,77,bottas,alfa,1:15.486,NaN,NaN
8005,8005,2022,21,interlagos,19,22,tsunoda,alphatauri,1:16.264,NaN,NaN
8006,8006,2022,21,interlagos,20,47,mick_schumacher,haas,1:16.361,NaN,NaN


# Driver Standings

In [19]:
driver_standings = {'season': [],
          'round': [],
          'position': [],
          'driverId': [],
          'constructorId': [],
          'points': [],
          'wins': []}

for index in rounds_df.index:
    for rnd_num in rounds_df['rounds'][index]:
        #print(rounds_df['season'][index],rnd_num)
        url = 'http://ergast.com/api/f1/{}/{}/driverStandings.json'
        r = requests.get(url.format(rounds_df['season'][index],rnd_num),timeout=None)
        json = r.json()
        try: 
            for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
                try:
                    driver_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
                except:
                    driver_standings['season'].append(None)

                try:
                    driver_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
                except:
                    driver_standings['round'].append(None)

                try:
                    driver_standings['position'].append(int(item['position']))
                except:
                    driver_standings['position'].append(None)

                try:
                    driver_standings['driverId'].append(item['Driver']['driverId'])
                except:
                    driver_standings['driverId'].append(None)

                try:
                    driver_standings['constructorId'].append(item['Constructors'][0]['constructorId'])
                except:
                    driver_standings['constructorId'].append(None)

                try:
                    driver_standings['points'].append(float(item['points']))
                except:
                    driver_standings['points'].append(None)

                try:
                    driver_standings['wins'].append(int(item['wins']))
                except:
                    driver_standings['wins'].append(None)
        except:
            print("Results not available yet")

Results not available yet


In [20]:
driver_standings_df = pd.DataFrame(driver_standings)
driver_standings_df.to_csv('driverStandings.csv')

In [24]:
driver_standings_df.tail()

,season,round,position,driverId,constructorId,points,wins
27553,2022,21,18,zhou,alfa,6.0,0
27554,2022,21,19,albon,williams,4.0,0
27555,2022,21,20,latifi,williams,2.0,0
27556,2022,21,21,de_vries,williams,2.0,0
27557,2022,21,22,hulkenberg,aston_martin,0.0,0


# Constructors Standings

In [33]:
constructor_standings = {'season': [],
          'round': [],
          'position': [],
          'constructorId': [],
          'points': [],
          'wins': []}

for index in rounds_df.index:
    for rnd_num in rounds_df['rounds'][index]:
        url = 'http://ergast.com/api/f1/{}/{}/constructorStandings.json'
        r = requests.get(url.format(rounds_df['season'][index],rnd_num),timeout=None)
        json = r.json()
        try: 
            for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
                try:
                    constructor_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
                except:
                    constructor_standings['season'].append(None)

                try:
                    constructor_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
                except:
                    constructor_standings['round'].append(None)

                try:
                    constructor_standings['position'].append(int(item['position']))
                except:
                    constructor_standings['position'].append(None)

                try:
                    constructor_standings['constructorId'].append(item['Constructor']['constructorId'])
                except:
                    constructor_standings['constructorId'].append(None)

                try:
                    constructor_standings['points'].append(float(item['points']))
                except:
                    constructor_standings['points'].append(None)

                try:
                    constructor_standings['wins'].append(int(item['wins']))
                except:
                    constructor_standings['wins'].append(None)
        except:
            pass

In [34]:
constructor_standings_df = pd.DataFrame(constructor_standings)
constructor_standings_df.to_csv('constructorStandings.csv')

In [35]:
constructor_standings_df.tail()

,season,round,position,constructorId,points,wins
12916,2022,21,6,alfa,55.0,0
12917,2022,21,7,aston_martin,50.0,0
12918,2022,21,8,haas,37.0,0
12919,2022,21,9,alphatauri,35.0,0
12920,2022,21,10,williams,8.0,0


# Drivers Information

In [9]:
driver_info = {'season': [],
              'driverId': [],
              'permanentNumber': [],
              'code': [],
              'givenName': [],
              'familyName': [],
              'dateOfBirth': [],
              'nationality': []}

for year in list(range(1950,2023)):
    url = 'http://ergast.com/api/f1/{}/drivers.json'
    r = requests.get(url.format(year),timeout=None)
    json = r.json()
    try:
        for item in json['MRData']['DriverTable']['Drivers']:
            try:
                driver_info['season'].append(int(json['MRData']['DriverTable']['season']))
            except:
                driver_info['season'].append(None)
                
            try:
                driver_info['driverId'].append(item['driverId'])
            except:
                driver_info['driverId'].append(None)
                
            try:
                driver_info['permanentNumber'].append(int(item['permanentNumber']))
            except:
                driver_info['permanentNumber'].append(None)
                
            try:
                driver_info['code'].append(item['code'])
            except:
                driver_info['code'].append(None)
                
            try:
                driver_info['givenName'].append(item['givenName'])
            except:
                driver_info['givenName'].append(None)
                
            try:
                driver_info['familyName'].append(item['familyName'])
            except:
                driver_info['familyName'].append(None)
                
            try:
                driver_info['dateOfBirth'].append(item['dateOfBirth'])
            except:
                driver_info['dateOfBirth'].append(None)
                
            try:
                driver_info['nationality'].append(item['nationality'])
            except:
                driver_info['nationality'].append(None)
                
    except:
        print("Results Not Available")

In [15]:
driver_info_df = pd.DataFrame(driver_info)
driver_info_df.to_csv('driverInfo.csv')

# Constructor Info

In [22]:
constructor_info = {'season': [],
              'constructorId': [],
              'name': [],
              'nationality': []}

for year in list(range(1950,2023)):
    url = 'http://ergast.com/api/f1/{}/constructors.json'
    r = requests.get(url.format(year),timeout=None)
    json = r.json()
    try:
        for item in json['MRData']['ConstructorTable']['Constructors']:
            try:
                constructor_info['season'].append(int(json['MRData']['ConstructorTable']['season']))
            except:
                constructor_info['season'].append(None)
                
            try:
                constructor_info['constructorId'].append(item['constructorId'])
            except:
                constructor_info['constructorId'].append(None)
                
            try:
                constructor_info['name'].append(item['name'])
            except:
                constructor_info['name'].append(None)
                
            try:
                constructor_info['nationality'].append(item['nationality'])
            except:
                constructor_info['nationality'].append(None)
    except:
        print("Results Not Available")

In [23]:
constructor_info_df = pd.DataFrame(constructor_info)
constructor_info_df.to_csv('constructorInfo.csv')

In [25]:
constructor_info_df.tail(10)

,season,constructorId,name,nationality
1081,2022,alfa,Alfa Romeo,Swiss
1082,2022,alphatauri,AlphaTauri,Italian
1083,2022,alpine,Alpine F1 Team,French
1084,2022,aston_martin,Aston Martin,British
1085,2022,ferrari,Ferrari,Italian
1086,2022,haas,Haas F1 Team,American
1087,2022,mclaren,McLaren,British
1088,2022,mercedes,Mercedes,German
1089,2022,red_bull,Red Bull,Austrian
1090,2022,williams,Williams,British


In [3]:
pd.read_csv('constructorInfo.csv').head()

,Unnamed: 0,season,constructorId,name,nationality
0,0,1950,adams,Adams,American
1,1,1950,alfa,Alfa Romeo,Swiss
2,2,1950,alta,Alta,British
3,3,1950,cooper,Cooper,British
4,4,1950,deidt,Deidt,American
